In [1]:
from bs4 import BeautifulSoup as bs
from numpy import nan
import seaborn as sns
import pandas as pd
import requests
from time import sleep
import matplotlib.pyplot as plt
import warnings
import folium
import datetime

#그래프 한글 오류 해결
from matplotlib import font_manager, rc

font_path = 'malgun.ttf'
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font_name)

#경고 메시지 무시
warnings.filterwarnings("ignore")

#마이너스 부호 유니코드
plt.style.use('ggplot')
plt.rcParams['axes.unicode_minus']=False

In [2]:
titanic = sns.load_dataset('titanic')
titanic.head(4)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False


In [3]:
def add_10(n):
    return n + 10

df = titanic.iloc[:5][['age','fare']]
df2 = df.apply(add_10)
df2

,age,fare
0,32.0,17.2500
1,48.0,81.2833
2,36.0,17.9250
3,45.0,63.1000
4,45.0,18.0500


,addr,kor,eng,math,sci,his
name,,,,,,
유재석,서초구,95,90,92,95,88
박명수,구로구,78,85,82,88,90
정준하,구로구,58,52,63,71,42
정형돈,종로구,88,90,85,95,82
노홍철,강남구,68,78,80,82,80
하하,강남구,78,80,80,78,75


In [4]:
def grade(score):
    if score >= 90 : grd = 'A'
    elif score >= 80 : grd = 'B'
    elif score >= 70 : grd = 'C'
    elif score >= 60 : grd = 'D'
    else: grd = 'F'
    return grd

students = pd.read_csv('csv/class_1.csv', encoding = 'euckr', header = None,
                      names = ['name','addr','kor','eng','math','sci','his'],
                      index_col = 'name')


students['avg'] = students.loc[:,'kor':'his'].mean(axis ='columns') #columns라고 써있지만 적용이 행마다 되어있다. 헷갈림 주의.
students['grade'] = students['avg'].apply(grade)
students

,addr,kor,eng,math,sci,his,avg,grade
name,,,,,,,,
유재석,서초구,95,90,92,95,88,92.0,A
박명수,구로구,78,85,82,88,90,84.6,B
정준하,구로구,58,52,63,71,42,57.2,F
정형돈,종로구,88,90,85,95,82,88.0,B
노홍철,강남구,68,78,80,82,80,77.6,C
하하,강남구,78,80,80,78,75,78.2,C


In [5]:
def add_three(x,y = 0,z = 0):
    return x+y+z
print(df['age'])
df['age'].apply(add_three, y = 10, z = 10)

print(df['age'].apply(lambda x:x+10))
print(df.applymap(lambda x:x+10))
#apply() : 한단계 낮은 차원의 단위별로 적용되는 함수. 데이터 프레임에 넣을시 결과는 시리즈
#
#applymap() : 데이터프레임의 원소 단위로 적용되는 함수. 데이터 프레임에 넣을시 결과는 데이터 프레임
def add_10(data):
    return data + 10
df.applymap(add_10)

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: age, dtype: float64
0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64
    age     fare
0  32.0  17.2500
1  48.0  81.2833
2  36.0  17.9250
3  45.0  63.1000
4  45.0  18.0500


,age,fare
0,32.0,17.2500
1,48.0,81.2833
2,36.0,17.9250
3,45.0,63.1000
4,45.0,18.0500


In [6]:
students.loc[:,'kor':'his'].applymap(grade)

,kor,eng,math,sci,his
name,,,,,
유재석,A,A,A,A,B
박명수,C,B,B,B,A
정준하,F,F,D,C,F
정형돈,B,A,B,A,B
노홍철,D,C,B,B,B
하하,C,B,B,C,C


In [7]:
# 시리즈 -> 시리즈
def missing_values(sr):
    return sr.isnull()

#시리즈 -> 값
def missing_values_count(sr):
    return sr.isnull().sum()

df = titanic.iloc[15:30][['age','fare']]
df.apply(missing_values, axis = 'index') #인덱스들을 묶어서 컬럼단위로 함수에 공급
df.apply(missing_values_count, axis = 'index')

age     5
fare    0
dtype: int64

In [8]:
df.apply(missing_values_count, axis = 'columns') #컬럼을 묶어서 인덱스 단위로 함수에 공급

15    0
16    0
17    1
18    0
19    1
20    0
21    0
22    0
23    0
24    0
25    0
26    1
27    0
28    1
29    1
dtype: int64

In [9]:
students = pd.read_csv('csv/class_1.csv', encoding = 'euckr', header = None,
                      names = ['name','addr','kor','eng','math','sci','his'],
                      index_col = 'name')
students

,addr,kor,eng,math,sci,his
name,,,,,,
유재석,서초구,95,90,92,95,88
박명수,구로구,78,85,82,88,90
정준하,구로구,58,52,63,71,42
정형돈,종로구,88,90,85,95,82
노홍철,강남구,68,78,80,82,80
하하,강남구,78,80,80,78,75


In [10]:
sr = students['kor']

def average(sr):
    return sr.sum() / len(sr.index)

students2 = students.copy()
students2['avg'] = students2.loc[:, 'kor':].apply(average, axis = 'columns')
students2

,addr,kor,eng,math,sci,his,avg
name,,,,,,,
유재석,서초구,95,90,92,95,88,92.0
박명수,구로구,78,85,82,88,90,84.6
정준하,구로구,58,52,63,71,42,57.2
정형돈,종로구,88,90,85,95,82,88.0
노홍철,강남구,68,78,80,82,80,77.6
하하,강남구,78,80,80,78,75,78.2


In [11]:
def add_two_obj(sr1, sr2):
    return sr1 + sr2

df2 = df.copy()
df2['ten'] = 10

# 데이터프레임(df2)를 컬럼(axis = 'columns')을 묶어서 행(index) 단위로 함수에 공급

#가상 시리즈 {age : 55.0, 'fare': 16.0, 'ten': 10} 생성하여 함수 적용됨
df2['age+ten'] = df2.apply(lambda sr: sr['age'] + sr['ten'], axis = 'columns')
df2

,age,fare,ten,age+ten
15,55.0,16.0000,10,65.0
16,2.0,29.1250,10,12.0
17,NaN,13.0000,10,NaN
18,31.0,18.0000,10,41.0
19,NaN,7.2250,10,NaN
20,35.0,26.0000,10,45.0
21,34.0,13.0000,10,44.0
22,15.0,8.0292,10,25.0
23,28.0,35.5000,10,38.0
24,8.0,21.0750,10,18.0


In [12]:
# 학생데이터(Series)가 공급되면 국어(kor), 영어(eng), 수학(math) 과목에 대한 합계를 구하는 함수
def sub3_average(sr):
    return (sr['kor'] + sr['eng'] + sr['math']) / 3

students2 = students.copy()
print(students2.apply(sub3_average, axis = 'columns'))

students2.apply(lambda sr:(sr['kor'] + sr['eng'] + sr['math']) / 3, axis = 'columns')

name
유재석    92.333333
박명수    81.666667
정준하    57.666667
정형돈    87.666667
노홍철    75.333333
하하     79.333333
dtype: float64


name
유재석    92.333333
박명수    81.666667
정준하    57.666667
정형돈    87.666667
노홍철    75.333333
하하     79.333333
dtype: float64

In [13]:
# 데이터프레임.pipe(함수) : 데이터프레임을 함수에 인자를 공급한다.
# 함수(데이터프레임)
def missing_value(df):
    return df.isnull()

def missing_count(df):
    return df.isnull().sum()

def missing_total_count(df):
    return df.isnull().sum().sum()


df = titanic.loc[15:24, ['age','fare','deck']]
missing_value(df)
missing_count(df)
missing_total_count(df)

10